# How People Decide what they want to do
Directed graph approach

Generally people want to do a number of different things. For this I'm going to create a schema for this in a graph language that allows me to designate how much a `pop` desires to take a certain action. This will be used later when determining AI decisions. 

**Note** this notebook actualy builds the desires into the graph, overwriting existing ontology. 

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f
import yaml, ssl, asyncio
import nb_black

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [2]:
local_user = "BillmanLocal2"
res = db.run_query(f"g.V().hasLabel('pop').has('username','{local_user}').valueMap()")
pops = [db.clean_node(n) for n in res]
pops[0]

{'conformity': 0.427,
 'literacy': 0.647,
 'aggression': 0.575,
 'constitution': 0.554,
 'objid': '9033676237817',
 'faction_no': 1,
 'name': 'Hyder Ranra',
 'isInFaction': '2731939737958',
 'industry': 0.5645,
 'wealth': 0.6058,
 'faction_loyalty': 0.4492,
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '9033676237817'}

Each population wants to do everything to a degree, the amount of desire to do that thing is expressed by the edge weight. 
* Attack a population
* Focus on improving literacy
* Focus on improving industry

# Desires as Objects

## Desire with targets.
Both factions and pops can have desire. Action is guided by desire based on the `max(desire.weight)`.
`desire` is an edge, the type of that desire is a property of that edge, and the edge weight is the amount of desire. The target (`node2`) is the recipient. 

Examples:
* faction wants trade with faction
* pop wants war with another pop
* pop wants faction to go to war with faction

## Desire without targets.

Desires without targets must link to an objective. That objective can be it's own node.

### This next cell will upload new desires from `desires.yaml`, which can be edited at any time. 


In [3]:
# # Drop the items, if they exist. 
# db.run_query("g.V().hasLabel('objective').has('username','notebook').drop()")
# objectives_yaml = yaml.safe_load(open("desires.yaml"))['objectives']
# data = {"nodes":objectives_yaml,'edges':[]}
# # Then Create the nodes and add them to the DB
# db.upload_data(data,verbose=False)
# After creating the nodes, pulling them into the notebook for reference
res = db.run_query("g.V().hasLabel('objective').valueMap()")
objectives = [db.clean_node(n) for n in res]
pd.DataFrame(objectives)


,type,weight,leadingAttribute,comment,username,objtype,objid,id
0,industry,0.5,wealth,"build factories, skyscrapers, infrastructure",notebook,objective,2008795542230,2008795542230
1,expansion,0.5,industry,"to increase population, eventually becoming a ...",notebook,objective,3832546566671,3832546566671
2,war,0.4,aggression,the general amount that the society wants to f...,notebook,objective,9111777612067,9111777612067
3,wealth,0.5,literacy,"increasing luxury, entertainiment and amenities",notebook,objective,9171875985828,9171875985828
4,science,0.5,literacy,"building schools, education systems, informati...",notebook,objective,7056154012146,7056154012146


# population wants to improve industry

populations want to improve industry when:
* they are not wealty
* they are at war




In [4]:
# Marginal return on base attribute
n = 2
ind_df = pd.DataFrame(np.sort([float(p['wealth']) for p in pops]),columns=['wealth'])
ind_df['base'] = range(len(ind_df))
ind_df['desires_industry'] = ind_df['wealth'].apply(lambda x: ((x+1)**(1-n) - 1)/(1-n))
ind_df['desire_base'] = ind_df['base'].apply(lambda x: ((x+1)**(1-n) - 1)/(1-n))
alt.Chart(ind_df).mark_line().encode(x='base',y='desire_base').properties(title="Desire relative to the base attribute")

alt.Chart(...)

In [5]:
alt.Chart(ind_df).mark_line().encode(x='wealth:N',y='desires_industry').properties(title="Desires wealth industry relative to industry")

alt.Chart(...)

## feeding that desire to the populations

Per above, the initial desire to do a thing is dependant on the `leading attribute`. This makes it easy to adjust the population desires in the `desires.yaml`.

In [6]:
def get_desire(x):
       return np.round(((float(x)+1)**(1-n) - 1)/(1-n),3)

popno = 0
objectiveno = 0

print(pops[popno])
print(objectives[objectiveno])

{'conformity': 0.427, 'literacy': 0.647, 'aggression': 0.575, 'constitution': 0.554, 'objid': '9033676237817', 'faction_no': 1, 'name': 'Hyder Ranra', 'isInFaction': '2731939737958', 'industry': 0.5645, 'wealth': 0.6058, 'faction_loyalty': 0.4492, 'username': 'BillmanLocal2', 'objtype': 'pop', 'id': '9033676237817'}
{'type': 'industry', 'weight': '0.5', 'leadingAttribute': 'wealth', 'comment': 'build factories, skyscrapers, infrastructure', 'username': 'notebook', 'objtype': 'objective', 'objid': '2008795542230', 'id': '2008795542230'}


In [7]:
get_desire(pops[popno][objectives[objectiveno]['leadingAttribute']])

0.377

Create a desire edge for each desire, for each population.

In [25]:
edges = []
for p in pops:
    for o in objectives:
        edge = {'label':'desires',
                'node1':p['objid'],
                'node2':o['objid'],
                'desire':o['type'],
                'weight':get_desire(p[o['leadingAttribute']])}
        edges.append(edge)

In [26]:
o

{'type': 'science',
 'weight': '0.5',
 'leadingAttribute': 'literacy',
 'comment': 'building schools, education systems, information infrastructure',
 'username': 'notebook',
 'objtype': 'objective',
 'objid': '7056154012146',
 'id': '7056154012146'}

In [27]:
pd.DataFrame(edges)

,label,node1,node2,desire,weight
0,desires,9033676237817,2008795542230,industry,0.377
1,desires,9033676237817,3832546566671,expansion,0.361
2,desires,9033676237817,9111777612067,war,0.365
3,desires,9033676237817,9171875985828,wealth,0.393
4,desires,9033676237817,7056154012146,science,0.393
5,desires,4278076177891,2008795542230,industry,0.385
6,desires,4278076177891,3832546566671,expansion,0.360
7,desires,4278076177891,9111777612067,war,0.310
8,desires,4278076177891,9171875985828,wealth,0.408
9,desires,4278076177891,7056154012146,science,0.408


In [11]:
db.create_edge(edges[0], local_user)

"g.V().has('objid','9033676237817').addE('desires').property('username','BillmanLocal2').property('weight',0.377).to(g.V().has('objid','2008795542230'))"

I'm not actually going to upload the edge, as this function is in production

In [12]:
# db.upload_data({'nodes':[],'edges':edges},verbose=False)

# Choosing based on the strongest desire

Here is how you take one pop, and calculate what it desires. 

In [13]:
def qtodf (query):
    res = db.run_query(query)
    nodes = [db.clean_node(n) for n in res]
    return pd.DataFrame(nodes)

query = "g.V().hasLabel('objective').valueMap()"
qtodf(query)

,type,weight,leadingAttribute,comment,username,objtype,objid,id
0,industry,0.5,wealth,"build factories, skyscrapers, infrastructure",notebook,objective,2008795542230,2008795542230
1,expansion,0.5,industry,"to increase population, eventually becoming a ...",notebook,objective,3832546566671,3832546566671
2,war,0.4,aggression,the general amount that the society wants to f...,notebook,objective,9111777612067,9111777612067
3,wealth,0.5,literacy,"increasing luxury, entertainiment and amenities",notebook,objective,9171875985828,9171875985828
4,science,0.5,literacy,"building schools, education systems, informati...",notebook,objective,7056154012146,7056154012146


In [14]:
qtodf(f"g.V().hasLabel('pop').has('username','{local_user}').limit(2).valueMap()")

,conformity,literacy,aggression,constitution,objid,faction_no,name,isInFaction,industry,wealth,faction_loyalty,username,objtype,id
0,0.408,0.797,0.727,0.220,1760394212974,0,Lano Ne,9823083597404,0.4735,0.6352,0.462,BillmanLocal2,pop,1760394212974
1,0.235,0.621,0.313,0.399,8528199481524,1,Jupi Ra,8607803573370,0.3560,0.4885,0.608,BillmanLocal2,pop,8528199481524


Arbitrarily grabbing a pop id: `9033676237817`

In [15]:
popid1 = "1760394212974"
popid2 = "8528199481524"

In [20]:
qtodf(f"g.V().has('objid','{popid1}').outE()")

KeyError: 0

In [17]:
qtodf(f"g.V().has('objid','{popid2}').out('desires').valueMap()")

,type,weight,leadingAttribute,comment,username,objtype,objid,id
0,expansion,0.5,industry,"to increase population, eventually becoming a ...",notebook,objective,3832546566671,3832546566671
1,industry,0.5,wealth,"build factories, skyscrapers, infrastructure",notebook,objective,2008795542230,2008795542230
2,science,0.5,literacy,"building schools, education systems, informati...",notebook,objective,7056154012146,7056154012146
3,wealth,0.5,literacy,"increasing luxury, entertainiment and amenities",notebook,objective,9171875985828,9171875985828
4,war,0.4,aggression,the general amount that the society wants to f...,notebook,objective,9111777612067,9111777612067


In [19]:
db.create_edge(edges[1], local_user)

"g.V().has('objid','9033676237817').addE('desires').property('username','BillmanLocal2').property('weight',0.361).to(g.V().has('objid','3832546566671'))"

In [32]:
edges[1]

{'label': 'desires',
 'node1': '9033676237817',
 'node2': '3832546566671',
 'weight': 0.361}